In [ ]:
import time as t
import numpy as np
import matplotlib.pyplot as plt

import qiskit
from qiskit.visualization import *
import qiskit_machine_learning as qml
from qiskit_machine_learning.connectors import TorchConnector

import torch
from torch import cat, no_grad, manual_seed
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau
from torch.nn import (
    Module,
    Conv2d,
    Linear,
    Dropout2d,
    CrossEntropyLoss,
    MaxPool2d,
    Flatten,
    Sequential,
    ReLU,
)
import torch.nn.functional as F

from utils import gtt, make_filt

In [ ]:
epochs = 10  # Set number of epochs
filt, digits = make_filt([0,1,3,4,8])

qubits = digits
n_train = 200*digits
n_test = int(n_train/10);

print(
f'using {qubits} Qubits @{n_train} datapoints: {filt} for {epochs} epochs'
)
train_loader, test_loader = gtt(n_train, filt)

In [ ]:
# Define and create QNN
def create_qnn(qubits):
    feature_map = qiskit.circuit.library.ZZFeatureMap(qubits)
    ansatz = qiskit.circuit.library.EfficientSU2(
                            qubits, su2_gates=['rx', 'ry', 'rz'], 
                            entanglement='circular', reps=1
    )
    qc = qiskit.circuit.QuantumCircuit(qubits)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)
    
#     print(f"Circuit Depth {qc.depth()}", qc)
    transpiled = qiskit.compiler.transpile(qc)
    print(transpiled)

    # REMEMBER TO SET input_gradients=True FOR ENABLING HYBRID GRADIENT BACKPROP
    qnn = qml.neural_networks.EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
    )
    return qnn

qnn4 = create_qnn(qubits)

In [ ]:
class Net(Module):
    def __init__(self, qnn):
        super().__init__()
        self.conv1 = Conv2d(1, 6, kernel_size=3)
        self.conv2 = Conv2d(6, 16, kernel_size=3)
        self.conv3 = Conv2d(16, qubits, kernel_size=3) # outputs 1x10
        self.dropout = Dropout2d()
        self.qnn = TorchConnector(qnn)  # Apply torch connector, weights chosen
        self.fc3 = Linear(1, 10) 

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)
        x = self.qnn(x)  # apply QNN
        x = self.fc3(x)
        return x; 

model4 = Net(qnn4)
print(model4)
print(model4(torch.randn(1,1,28,28))) # Just testing

In [ ]:
# Define model, optimizer, and loss function
optimizer = Adam(model4.parameters(), lr=0.01)
scheduler = ExponentialLR(optimizer, gamma=0.9)

loss_func = CrossEntropyLoss()

# Start training
loss_list = [2.3]  # Store loss history
model4.train()  # Set model to training mode

itrs = len(train_loader)
logspan = int(itrs*12/100) # 12%

print(f"Running training for {qubits} Qubits @{itrs} itrs/epoch")

In [ ]:
for epoch in range(epochs):
    total_loss = []
    times = []
    now = t.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        z = t.time()
        optimizer.zero_grad(set_to_none=True)  # Initialize gradient
        output = model4(data)  # Forward pass
        loss = loss_func(output, target)  # Calculate loss
        loss.backward()  # Backward pass
        
        optimizer.step()  # Optimize weights
        total_loss.append(loss.item())  # Store loss
        z = t.time() - z;
        if ((batch_idx%(logspan))==0): print(f"{int(z)*logspan} sec/{logspan}itrs")
    
    scheduler.step()
    end = int((t.time() - now)/60)+1
    loss_list.append(sum(total_loss) / len(total_loss))
    print("Trained [{:.0f}%]\tLoss: {:.4f}".format(100.0 * (epoch + 1) / epochs, loss_list[-1]), 
          f"in {end} min \t(est. {int((epochs-epoch)*end)} min left)")
    
    diff = np.abs(loss_list[-1] - loss_list[-2]) /loss_list[-1]; 
    if diff <= 0.0005: # Early stopping criterial loss diff = 0.1%
        print("Τraining Complete")
        break;

In [ ]:
# Plot loss convergence
plt.plot(loss_list)
plt.title("Hybrid NN Training Convergence")
plt.xlabel("Training Iterations")
plt.ylabel("Neg. Log Likelihood Loss")
plt.show()

In [ ]:
model4.eval()
with torch.no_grad():
    
    correct = 0
    for (data, target) in test_loader:
        output = model4(data)
        
        pred = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        loss = loss_func(output, target)
        total_loss.append(loss.item())
        
    print('Performance on test data:\n\tLoss: {:.4f}\n\tAccuracy: {:.1f}%'.format(
                sum(total_loss) / len(total_loss),
                correct / len(test_loader) * 100)
            )
    print(f"Perfectly Random would be {int(100/digits)}%")

In [ ]:
# Plot predicted labels

n_samples_show = 15
count = 0
fig, axes = plt.subplots(nrows=1, ncols=n_samples_show, figsize=(10, 3))

model4.eval()
with no_grad():
    for (data, target) in test_loader:
        if count == n_samples_show:
            break
        output = model4(data[0:1])
        if len(output.shape) == 1:
            output = output.reshape(1, *output.shape)

        pred = output.argmax(dim=1, keepdim=True)

        axes[count].imshow(data[0].numpy().squeeze(), cmap="viridis")

        axes[count].set_xticks([])
        axes[count].set_yticks([])
        axes[count].set_title("{}".format(pred.item()))

        count += 1

In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
html{filter:invert(0.86)}

div.prompt{opacity: 0.5;}

.btn-default{border-color: transparent;}

#header-container{display:none !important;}

div.cell.selected, div.cell.selected.jupyter-soft-selected{border-color: transparent;}
</style>
""")